#### Representation Learning for Dynamic 3D Scenes

3D scene representations in various forms depict the environment at diﬀerent levels of
abstraction, implying distinct modelling and generalisation capabilities. Therefore, it is essential to understand their strengths and weaknesses to choose the most appropriate one for the task at hand. In this
chapter,we explore three primary 3D representations: **keypoints**, parti**c**les, and **neural fields**, along with their corresponding model classes
for dynamic modelling of the underlying 3D world.

We'll focus
on how to incorporate external actions as input,predic future system
evolution, and formulate planning and control problems using the
learned dynamic models.

[Keypoints into the Future: Self-Supervised Correspondence in Model-Based Reinforcement Learning](https://arxiv.org/abs/2009.05085) proposed using 3D object key-
points, which are learned in a self-supervised manner and tracked
over time. These keypoints model the underlying dynamic 3D scenes
through model-based predictions, and with concrete experimental
evidence, they demonstrated that keypoints oﬀer a range of appealing properties:
- the output is interpretable and in 3D space, allowing analysis of the visual model separately from the predictive model,
- they can be applied to deformable objects
- they achieve category-level generalisation.

To provide a more concrete illustration of the problem and eﬀectively
showcase the advancements of the methods discussed in this notebook,
we outline **the formulation of the dynamics model representing the underlying 3D world**. We also explain how to learn the model from
data and how to formulate a planning problem using the dynamics
model.

The objective is to learn a dynamic model of the 3D environment
that predicts how the environment will change when an agent applies
a specific *action*.

##### Perception Module:

the first step contains a perception module $z_t = g(y_{t′:t},u_{t′:t−1})$ maps a small
sequence of observations $y_{t′:t}$ from time $t′$ to the current time $t$ and
the corresponding action $u_{t′:t−1}$ into a **latent representation of the environment $z_t$ at time $t$**.

##### Dynamics Model ($f_{\theta}$)

Once we obtain the scene representation $z_t$, we learn the **dynamics model $f_{\theta}$, parameterised by ${\theta}$, as a proxy for the real dynamics**. This
the new representation as a result of the action at time $t+1$
model takes the representation $z_t$ and action $u_t$ as inputs to predict

$$z_{t+1} = f_\theta(z_t,u_t)$$

##### Loss and Optimization:

$$\mathcal{L}(\theta) = \sum_{t} \left\| z^{*}_{t+1} - f_{\theta}(z_t, u_t) \right\|_{2}^{2}.$$

where $z^{*}$ is the actual representation of the state

The problem definition includes three critical constraints: 
- the **perception module** that maps the observations to a scene representation $z_t$, 
- the **dynamic model** that predicts the underlying system’s evolution,
- a **control module** that derives the control signal based on the current scene representation.

A **differentiable renderer** is a rendering system where the rendering process—from 3D scene parameters (e.g., shape, texture, lighting, and camera pose) to the final 2D image—is differentiable with respect to its inputs. This means you can compute gradients of a loss function (e.g., pixel-wise image difference) with respect to the 3D scene parameters. These gradients can be backpropagated through the renderer, allowing end-to-end gradient-based optimization or learning, typically with deep learning.

[this paper](https://arxiv.org/pdf/1906.11883)proposed learning neural implicit representations of 3D shape and appearance, supervised
with posed 2D images via a **diﬀerentiable renderer**

To leverage the 3D representations for modelling the dynamic
changes in the scene, researchers have explored object-level deformation through the lens of 3D keypoints [Ref: KeyPointsDeformer](https://arxiv.org/pdf/1906.11883) or meshes. Scene-level deformation can be modelled by *transporting* input coordinates to
neural implicit representations with an implicitly represented flow
field or time-variant latent code [Ref: Space-time Neural Irradiance Fields for Free-Viewpoint Video](https://arxiv.org/pdf/2011.12950)

----
In the works presented here, we explore methods to
incorporate external actions into learned dynamics models. This
approach enables the handling of various initial conditions and input
actions, making them more suitable for synthesising control signals
for embodied AI agents.

----

##### **Latent-Space Dynamics Model**

Instead of modeling the dynamics directly in the raw observation space (e.g., images, point clouds, sensor data), we first encode the observations into a compact latent space using an encoder network (like a *CNN*, *RNN*, or *Transformer*), and then model the dynamics in that latent space.

$$\textbf{Observation} \xrightarrow{\text{Encoder}} z_t \quad \text{(latent vector)}$$

$$z_{t+1} = f(z_t, a_t) \quad \text{(latent dynamics model)}$$

- $z_t$ is a latent (compressed, abstract) representation of the state.
- $f$ is a neural network that models how the system evolves in this abstract space.

The primary contribution of the presented method is (i) a novel
formulation of *predictive model-learning*, using learned self-
supervised 3D keypoints as the state representation. The learned
model can then be used to perform closed-loop visual feedback control via model-predictive control **(MPC)**. (ii) Through simulated
manipulation experiments, we show that the presented method out-
performs a variety of baselines, and (iii) we validate the approach in
real-world robot experiments.

dense-correspondence model and visual-correspondence models:
Dense correspondence models are neural networks (usually deep CNNs or transformers) trained to predict per-pixel correspondences between images.

First, we provide a brief background on dense correspondence models.Given an image observation $y_t \in \mathbb{R}^{W \times H \times C}$ (where $C$ denotes the number of channels), the dense-correspondence model $g^{dc}$ produces a full-resolution descriptor image $I_D \in \mathbb{R}^{W \times H \times D}$. Since the aim is to learn a dynamics model on a low-dimensional state, we need a way to construct $z_t$ from the descriptor image $I_D$. The idea is for $z_t$ to be a set of points on the object(s) that are localised in either image space or 3D space. These points are represented as a set $\{ d^k \}_{k=1}^K$ of $K$ descriptors, where each $d^k \in \mathbb{R}^D$ is a vector in the underlying descriptor space. A parameterless correspondence function $g^{\text{corr}}(I_D, d^k)^a$ determines the location of the keypoint $z_t^k \in \mathbb{R}^B$ from the current observation. By combining our learned keypoint correspondences with the reference descriptors, we have a function that maps image observations $y_t$ to keypoint locations $z_t^{\text{object}} = \{ z_t^k \}_{k=1}^K$ on the object(s) of interest. We present two methods for constructing the latent state $z_t = g(y_t)$ from $y_t$.

1. Descriptor set (DS)
2. Spatial descriptor set (SDS)

[**MPPI**](https://arxiv.org/pdf/1509.01149): 

**MPPI is a gradient-free optimiser that takes into account coordination between time steps when sampling action trajectories.** The algorithm samples $N$ trajectories, rolls them out using the trained model, calculates the reward/cost for each trajectory, and then re-weights the trajectories to sample a new set. A single trajectory in the MPC’s look-ahead horizon $H$ consists of state-action pairs $\{(z_t, u_t)\}_{t=0}^H$. Let $R^{(k)} = \sum_{t=0}^H r(z_t^{(k)}, u_t^{(k)})$ represent the reward of the $k_{th}$ trajectory, which could be the negation of the cost $c(z_t^{(k)}, u_t^{(k)})$. Define

$$
\mu_t = \frac{\sum_{k=1}^N \left(e^{\gamma R^{(k)}}\right) u_t^{(k)}}{\sum_{k=1}^N e^{\gamma R^{(k)}}}, \quad \forall t \in \{0, \ldots, H\}.
$$

A filtering technique is then employed to sample a new batch of trajectories from the previously computed mean $\mu_t$, specifically

$$
u_t^{(k)} = u_t^{(k)} + \mu_t,
$$

where the noise $\eta_t^{(k)}$ is sampled as

$$
\alpha_t^{(k)} \sim \mathcal{N}(0, \Sigma), \quad \forall k \in \{1, \ldots, N\}, \quad \forall t \in \{0, \ldots, H\},
$$

$$
\eta_t^{(k)} = \beta \alpha_t^{(k)} + (1 - \beta) \eta_{t<0}^{(k)}, \quad \text{where} \quad \eta_{t<0} = 0.
$$

This process is repeated for $M$ iterations, after which the best action sequence is chosen. The key to the success of the method is leveraging GPUs for parallel sampling and evaluations.

In [2]:
# example of Spatial Descriptor Set (SDS) computation
import numpy as np
from sklearn.neighbors import NearestNeighbors

# 1. Generate a toy 3D point cloud
np.random.seed(42)
num_points = 100
points = np.random.rand(num_points, 3)  # 100 points in 3D space

# 2. Initialize Nearest Neighbors Search
k_neighbors = 10  # How many neighbors we look at
nbrs = NearestNeighbors(n_neighbors=k_neighbors, algorithm='auto').fit(points)

# 3. Compute descriptors
descriptors = []

for point in points:
    # Find k nearest neighbors (including the point itself)
    distances, indices = nbrs.kneighbors([point])
    
    # Get the neighbor points
    neighbors = points[indices[0]]
    
    # 4. Compute a simple descriptor:
    # Example descriptor: [mean(xyz of neighbors), std(xyz of neighbors)]
    mean_xyz = np.mean(neighbors, axis=0)      # Mean of neighbors
    std_xyz = np.std(neighbors, axis=0)         # Standard deviation of neighbors
    
    # Concatenate into a single descriptor vector
    descriptor = np.concatenate([mean_xyz, std_xyz])  # Shape (6,)
    
    # Add to the descriptor set
    descriptors.append(descriptor)

# 5. Convert to numpy array
SDS = np.array(descriptors)  # Shape (num_points, 6)

print(f"SDS shape: {SDS.shape}")  # (100, 6)
print(f"First descriptor example:\n{SDS[0]}")
print(f"Second descriptor example:\n{SDS[1]}")

SDS shape: (100, 6)
First descriptor example:
[0.39276933 0.86833655 0.77039301 0.14633547 0.10117541 0.13483603]
Second descriptor example:
[0.64830878 0.20331995 0.17095737 0.10893815 0.14273669 0.07584098]


$?$ one important note about heatmaps:

In dynamic 3D scenes, you often encode motion information into the **heatmap** across time:
So at time $t+1$, you expect the heatmap to shift according to the motion of the object!

#### contrastive loss definition:

$$\text{sim}(f_1, f_2) = \frac{f_1^\top f_2}{\|f_1\| \|f_2\|}$$

And the loss (InfoNCE) is:

$$\mathcal{L} = -\log\left( \frac{\exp(\text{sim}(q, k^+)/\tau)}{\sum_{k \in \{k^+, k^-\}} \exp(\text{sim}(q, k)/\tau)} \right)$$

where:
- $q$ = query feature
- $k^+$ = positive key
- $k^-$ = negative keys
- $\tau$ = temperature hyperparameter (controls hardness)


**Heatmap loss**: 

Let $p^*$ be the pixel space location of a ground-truth match. Then we can define the ground-truth heatmap as
$$H^(p) = \exp\left(-\frac{|p - p^*|^2}{\sigma^2}\right),$$
where $p$ represents a pixel location. A predicted heatmap can be obtained from the descriptor image $\mathcal{I}_D$ together with a reference descriptor $d^*$. Then the predicted heatmap is gotten by
$$\hat{H}(p; d^, \mathcal{I}_D, \eta) = \exp\left(-\frac{|\mathcal{I}_D(p) - d^*|^2}{\eta^2}\right).$$
The heatmap can also be normalised to sum to one, in which case it represents a probability distribution over the image:
$$\tilde{H}(p) = \frac{\hat{H}(p)}{\sum_{p' \in \Omega} \hat{H}(p')}.$$
The heatmap loss is simply the MSE between $H^*$ and $\hat{H}$ with mean reduction:
$$L_{\text{heatmap}} = \frac{1}{|\Omega|} \sum_{p \in \Omega} |\hat{H}(p) - H^*(p)|^2.$$


**Spatial expectation loss**: 

Given a descriptor $d^*$ together with a descriptor image $\mathcal{I}_D$, we can compute the 2D spatial expectation as
$$J_{\text{pixel}}(d^, \mathcal{I}D, \eta) = \sum{p \in \Omega} p \cdot \hat{H}(p; d^, \mathcal{I}_D, \eta).$$
If we also have a depth image $D$, then we can define the spatial expectation over the depth channel as
$$J_{\text{depth}}(d^,\mathcal{I}D,D,\eta) = \sum{p \in \Omega} D(p) \cdot \hat{H}(p;d^,\mathcal{I}_D,\eta).$$
The spatial expectation loss is simply the L1 loss between the ground truth and estimate corresponding using
$$L_{\text{spatial pixel}} = \left| p^* - J_{\text{pixel}}(d^*) \right|_1.$$
We can also use our 3D spatial expectation $J_{\text{depth}}$ to compute a 3D spatial expectation loss. In particular, given a depth image $D$, let the depth value corresponding to pixel $p$ be denoted by $D(p)$. The spatial expectation loss is simply
$$L_{\text{spatial depth}} = \left| D(p^) - J_{\text{depth}}(d^,\mathcal{I}_D,D,\eta) \right|_1.$$
Be careful to only take the expectation over pixels with valid depth values $D(p)$.
Total loss: The total loss is the weighted sum of the heatmap loss and the spatial loss:
$$L = w_{\text{heatmap}} L_{\text{heatmap}} + w_{\text{spatial}} (L_{\text{spatial pixel}} + L_{\text{spatial depth}}),$$
where the $w$ are weights.
Correspondence function: The correspondence function $g^{\text{corr}}(\mathcal{I}D,d^k)$ in Section 4.2.1 is defined using the spatial expectations $J{\text{pixel}},J_{\text{depth}}$ defined earlier to localise the descriptor $d^k$ in either pixel space or 3D space. If in 3D, we additionally use the known camera extrinsics to express the localised point in the world frame.
